<a href="https://colab.research.google.com/github/igorcduarte/AI-Multi-agents-for-Business/blob/main/AI_Multi_agents_for_Business_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
"""
Sistema Multiagente para Gerenciamento Contábil e Tributário de Empresas

Objetivo: Auxiliar empresas a gerirem questões contábeis, tributárias e de planejamento financeiro
por meio de uma arquitetura multiagente. Inclui visualizações otimizadas no Google Colab.
"""
%pip -q install google-genai

In [2]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 925.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar para interação com agentes
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [14]:
# Agente 1: Coletor de Dados Empresariais
def agente_coletor(tipo_negocio, localizacao, faturamento, despesas):
    coletor = Agent(
        name="agente_coletor",
        model="gemini-2.0-flash",
        description="Coleta informações sobre o negócio do usuário.",
        tools=[],
        instruction="""
        Você é um consultor especializado em coleta de informações para análise de negócios.
        Pergunte ao usuário sobre o tipo de negócio, localização, faturamento estimado e despesas mensais.
        Retorne os dados estruturados para uso posterior.
        """
    )
    entrada = f"Tipo de negócio: {tipo_negocio}\nLocalização: {localizacao}\nFaturamento: {faturamento}\nDespesas: {despesas}"
    return call_agent(coletor, entrada)

In [15]:
# Agente 2: Calculador Tributário
def agente_calculador(faturamento, regime_tributario):
    calculador = Agent(
        name="agente_calculador",
        model="gemini-2.0-flash",
        description="Calcula os impostos baseados no faturamento e no regime tributário.",
        tools=[],
        instruction="""
        Você é um especialista em tributação. Com base no regime tributário fornecido e no faturamento mensal,
        calcule os impostos devidos e forneça uma explicação detalhada para o usuário.
        """
    )
    entrada = f"Faturamento: {faturamento}\nRegime tributário: {regime_tributario}"
    return call_agent(calculador, entrada)

In [16]:
# Agente 3: Consultor Legal
def agente_legal(tipo_negocio, localizacao):
    consultor = Agent(
        name="agente_legal",
        model="gemini-2.0-flash",
        description="Orienta sobre registro e obrigações legais de empresas.",
        tools=[google_search],
        instruction="""
        Você é um consultor jurídico especializado em leis empresariais locais. Forneça ao usuário
        um guia passo a passo para registro da empresa, além de listar as obrigações fiscais relevantes
        baseadas no tipo de negócio e na localização fornecida.
        """
    )
    entrada = f"Tipo de negócio: {tipo_negocio}\nLocalização: {localizacao}"
    return call_agent(consultor, entrada)

In [17]:
# Agente 4: Planejador de Fluxo de Caixa
def agente_planejador(faturamento, despesas):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description="Auxilia na organização financeira mensal.",
        tools=[],
        instruction="""
        Você é um planejador financeiro. Baseado no faturamento e nas despesas fornecidas, crie um
        plano de fluxo de caixa que inclua entradas, saídas e uma estimativa de lucro líquido mensal.
        """
    )
    entrada = f"Faturamento: {faturamento}\nDespesas: {despesas}"
    return call_agent(planejador, entrada)

In [18]:
# Agente 5: Otimizador de Despesas
def agente_otimizador(despesas):
    otimizador = Agent(
        name="agente_otimizador",
        model="gemini-2.0-flash",
        description="Identifica áreas de economia nas despesas do negócio.",
        tools=[],
        instruction="""
        Você é um especialista em otimização financeira. Revise as despesas fornecidas e sugira
        cortes ou ajustes que podem ajudar o usuário a economizar.
        """
    )
    entrada = f"Despesas mensais: {despesas}"
    return call_agent(otimizador, entrada)

In [19]:
# Agente 6: Simulador de Crescimento
def agente_simulador(faturamento, despesas):
    simulador = Agent(
        name="agente_simulador",
        model="gemini-2.0-flash",
        description="Projeções de crescimento baseado em diferentes cenários.",
        tools=[],
        instruction="""
        Você é um especialista em análise de crescimento empresarial. Baseado no faturamento e nas despesas,
        forneça projeções de crescimento para 6 meses e 1 ano em diferentes cenários (conservador, moderado e otimista).
        """
    )
    entrada = f"Faturamento: {faturamento}\nDespesas: {despesas}"
    return call_agent(simulador, entrada)

In [20]:
# Agente 7: Revisor de Planejamento
def agente_revisor(plano):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Revisa e sugere ajustes no plano fornecido.",
        tools=[],
        instruction="""
        Você é um especialista em revisão de planejamento financeiro. Analise o plano fornecido, indique
        possíveis pontos fracos e sugira melhorias que possam fortalecer o planejamento empresarial.
        """
    )
    entrada = f"Plano fornecido: {plano}"
    return call_agent(revisor, entrada)

In [23]:
# Agente 8: Analista de Mercado
def agente_mercado(tipo_negocio, localizacao):
    mercado = Agent(
        name="agente_mercado",
        model="gemini-2.0-flash",
        description="Fornece análise de mercado para novos negócios.",
        tools=[google_search],
        instruction="""
        Você é um especialista em análise de mercado. Baseado no tipo de negócio e na localização fornecida,
        forneça uma visão geral do mercado, incluindo oportunidades, concorrência e demanda esperada.
        """
    )
    entrada = f"Tipo de negócio: {tipo_negocio}\nLocalização: {localizacao}"
    return call_agent(mercado, entrada)

In [24]:
# Agente 9: Orientador de Regime Tributário
def agente_regime(tipo_negocio):
    regime = Agent(
        name="agente_regime",
        model="gemini-2.0-flash",
        description="Orienta na escolha do regime tributário adequado.",
        tools=[],
        instruction="""
        Você é um especialista tributário. Com base no tipo de negócio, recomende o regime tributário mais adequado (Simples, Lucro Presumido, etc.).
        Explique os motivos da escolha.
        """
    )
    entrada = f"Tipo de negócio: {tipo_negocio}"
    return call_agent(regime, entrada)

In [25]:
# Agente 10: Orientador Sobre Resgistro da Empresa
def agente_registro(tipo_negocio, localizacao):
    registro = Agent(
        name="agente_registro",
        model="gemini-2.0-flash",
        description="Fornece um guia para registro de novas empresas.",
        tools=[google_search],
        instruction="""
        Você é um especialista em registro empresarial. Forneça um passo a passo detalhado para registrar uma empresa
        do tipo fornecido na localização especificada, incluindo custos e documentação necessária.
        """
    )
    entrada = f"Tipo de negócio: {tipo_negocio}\nLocalização: {localizacao}"
    return call_agent(registro, entrada)

In [29]:
def main():
    tem_empresa = input("Você já tem uma empresa? (sim/não): ").strip().lower()

    if tem_empresa == "sim":
        print("\n--- Gestão de Empresa Existente ---\n")
        tipo_negocio = input("Informe o tipo de negócio: ")
        localizacao = input("Informe a localização da empresa: ")
        faturamento = input("Informe o faturamento estimado mensal: ")
        despesas = input("Informe as despesas mensais estimadas: ")

        print("\n--- Dados Coletados ---\n")
        dados_coletados = agente_coletor(tipo_negocio, localizacao, faturamento, despesas)
        print(dados_coletados)

        print("\n--- Cálculo Tributário ---\n")
        regime_tributario = input("Informe o regime tributário (Simples, Lucro Presumido, etc.): ")
        calculo_impostos = agente_calculador(faturamento, regime_tributario)
        print(calculo_impostos)

        print("\n--- Consultoria Legal ---\n")
        consultoria = agente_legal(tipo_negocio, localizacao)
        print(consultoria)

        print("\n--- Planejamento Financeiro ---\n")
        plano_financeiro = agente_planejador(faturamento, despesas)
        print(plano_financeiro)

        print("\n--- Otimização de Despesas ---\n")
        otimizacao = agente_otimizador(despesas)
        print(otimizacao)

        print("\n--- Projeção de Crescimento ---\n")
        simulacao = agente_simulador(faturamento, despesas)
        print(simulacao)

        print("\n--- Revisão do Planejamento ---\n")
        revisao = agente_revisor(plano_financeiro)
        print(revisao)

    elif tem_empresa == "não":
        print("\n--- Criação de Nova Empresa ---\n")
        tipo_negocio = input("Qual tipo de negócio você deseja iniciar? ")
        localizacao = input("Em qual localização você pretende abrir a empresa? ")

        print("\n--- Análise de Mercado ---\n")
        analise_mercado = agente_mercado(tipo_negocio, localizacao)
        print(analise_mercado)

        print("\n--- Escolha do Regime Tributário ---\n")
        recomendacao_regime = agente_regime(tipo_negocio)
        print(recomendacao_regime)

        print("\n--- Guia de Registro da Empresa ---\n")
        guia_registro = agente_registro(tipo_negocio, localizacao)
        print(guia_registro)

    else:
      print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()


Você já tem uma empresa? (sim/não): sim

--- Gestão de Empresa Existente ---

Informe o tipo de negócio: TI
Informe a localização da empresa: Trindade
Informe o faturamento estimado mensal: 50000
Informe as despesas mensais estimadas: 20000

--- Dados Coletados ---

```json
{
  "tipo_de_negocio": "TI",
  "localizacao": "Trindade",
  "faturamento_estimado": 50000,
  "despesas_mensais": 20000
}
```


--- Cálculo Tributário ---

Informe o regime tributário (Simples, Lucro Presumido, etc.): Simples
Com base no seu faturamento de R$50.000 e no regime tributário Simples Nacional, o cálculo dos impostos será feito conforme as tabelas e alíquotas estabelecidas para este regime. No Simples Nacional, o valor dos impostos depende do seu faturamento acumulado nos últimos 12 meses e da sua atividade (CNAE).

**Passo 1: Enquadramento nas Tabelas do Simples Nacional**

O Simples Nacional possui diferentes tabelas (Anexos) dependendo da atividade da empresa. As tabelas variam de acordo com o setor (Co